In [ ]:
%pip install datasets pandas pymongo sentence_transformers
%pip install -U transformers
%pip install accelerate

In [ ]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("hhe1ibeb/xinyi_geodata")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

In [ ]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset="description")
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

dataset_df.head(5)

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

In [ ]:
dataset_df["embedding"] = dataset_df["description"].apply(get_embedding)

dataset_df.head()

## Connect to MongoDB

In [ ]:
import pymongo
import os

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = os.getenv("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

In [ ]:

# Ingest data into MongoDB
db = mongo_client["xinyi_geodata"]
collection = db["collection_1"]
# Delete any existing records in the collection
# collection.delete_many({})

In [ ]:
from pydantic import BaseModel

# Define Schema
class Coord(BaseModel):
    lat: float
    lon: float
    description: str


async def get_coord(lat: float, lon: float):
    item = await collection.find_one({"lat":lat, "lon":lon})
    if item:
        return item
    return None

get_coord(25.03334, 121.58122)

In [ ]:
documents = dataset_df.to_dict("records")
# collection.insert_many(documents)

# print("Data ingestion into MongoDB completed")

In [ ]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "lat": 1,  # Include the lat field
                "lon": 1,  # Include the lon field
                "description": 1,  # Include the description field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [ ]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Lat: {result.get('lat', 'N/A')}, Lon: {result.get('lon', 'N/A')}, Description: {result.get('description', 'N/A')}\n"

    return search_result

In [ ]:
# Conduct query with retrival of sources
query = "I want to find a place where I get a lot of stores nearby"
source_information = get_search_result(query, collection)
combined_information = (
    f"Query: {query}\nAccording to the results, suggest the best place in response to the query:\n{source_information}."
)

print(combined_information)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [ ]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to('cuda')
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))